In [1]:
import numpy as np
from tqdm import tqdm
import sys
from keras import models
from keras import layers
sys.path.append('utils/')

from keras.models import Model

Using TensorFlow backend.


In [2]:
train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

print(train_generator)

Found 1233 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [4]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

model.load_weights('teacher_weights.hdf5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 128)      

In [5]:
# remove softmax
model.layers.pop()
model = Model(model.input, model.layers[-1].output)
# now model outputs logits

In [6]:
from tqdm import tqdm
import numpy as np

batches = 0
train_logits = {}

for x_batch, name_batch in tqdm(train_generator):
    #print(np.shape(x_batch))
    #print(name_batch)
    #print(len(x_batch))
    #print(x_batch)
    
    batch_logits = model.predict_on_batch(x_batch)
    
    '''
    # Checking
    
    print(np.shape(batch_logits))
    #print(batch_logits)
    for i, n in enumerate(name_batch):
        print(batch_logits[i])
        print(n)
    '''
    
    for i, n in enumerate(name_batch):
        #print(i)
        #print(n)
        #print(train_logits)
        #print(batch_logits[i])
        train_logits[n] = batch_logits[i]

  0%|                                                                                           | 0/62 [00:00<?, ?it/s]

(20, 299, 299, 3)


  0%|                                                                                           | 0/62 [00:02<?, ?it/s]

(20, 10)
[9.9999404e-01 4.6660030e-06 6.1792473e-09 1.7965229e-08 5.5401091e-07
 1.1006326e-13 5.8776681e-07 1.0414213e-11 8.1122067e-08 1.3741929e-12]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.5648283e-04 2.1063967e-03 5.1945648e-05 4.7695744e-07 5.8945457e-08
 3.7551521e-11 2.2951220e-07 9.9756819e-01 1.4652124e-05 1.6847829e-06]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[6.1069396e-03 1.5808641e-03 1.1079933e-02 9.7151542e-01 3.0115894e-03
 1.0576836e-03 8.4964046e-04 5.3919513e-05 3.6088272e-03 1.1352485e-03]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[1.2698478e-08 1.5972486e-04 9.9978071e-01 2.2829943e-06 7.2422329e-10
 5.8703620e-08 1.0723579e-05 4.6174282e-05 3.2977377e-07 1.7632200e-09]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1.1793785e-05 5.1060006e-06 9.9559021e-01 7.4108408e-07 8.4767350e-07
 8.2210983e-07 1.7488235e-06 4.1704443e-03 2.1830961e-04 1.2741569e-07]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[4.9386449e-07 1.1084245e-04 1.0550124e-06 3.8726392e-04 9.2518756e-05
 9.9810696e-01 5.7433205e-09 5.1701656e-05 7.

TypeError: unhashable type: 'numpy.ndarray'